**Задание**
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации

2. Рассмотреть 2-а варианта сеточек 

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

 				
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [63]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [65]:
max_words = 10000
max_len = 40
num_classes = 1
EMBEDDING_DIM = 300
# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [66]:
data = pd.read_excel('data.xls')

In [67]:
data = data[(data['Rating'] != 3)]

In [68]:
data['Rating'] = (data['Rating'] > 3).astype(int)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data['Content'], data['Rating'], test_size=0.33, random_state=42)

#### Подготовим текст, уберем стопслова, пунктуацию, приведем слова к нормальной форме.

In [70]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)


In [71]:
X_train = X_train.apply(preprocess_text)

In [72]:
X_test = X_test.apply(preprocess_text)

#### Соберем весть трейн сет в корпус слов (склеим все строки) и разобьем на токены.

In [73]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [74]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Отфильтруем данные и соберём в корпус N наиболее частых токенов (параметр max_words)

In [75]:
tokens_filtered = [word for word in tokens if word.isalnum()]  # Фильтруем, чтобы не было чисел в токенах, т.к. они бесполезны

In [76]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [77]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'супер',
 'телефон']

In [78]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()} # Делаем словарь типа слово:индекс слова

#### Переводим текст в массив нампи, где значение ячейки = индексу слова. Длина строки - параметр max_len

In [79]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [80]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [81]:
X_train[4]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  308,    1, 2206], dtype=int32)

#### Создаем модель на Keras

In [82]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import BinaryCrossentropy 
from keras.callbacks import EarlyStopping  

In [83]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [84]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [85]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 13s 477ms/step - loss: 0.4831 - accuracy: 0.8178 - val_loss: 0.3788 - val_accuracy: 0.8573
Epoch 2/10
24/24 [==============================] - 11s 466ms/step - loss: 0.3226 - accuracy: 0.8674 - val_loss: 0.2447 - val_accuracy: 0.9094


In [86]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 1s 80ms/step


In [87]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [88]:
results_tresh = (results > 0.5) * 1

In [89]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.74      0.59      0.65      1029
           1       0.93      0.96      0.94      5488

    accuracy                           0.90      6517
   macro avg       0.83      0.77      0.80      6517
weighted avg       0.90      0.90      0.90      6517



In [90]:
print(roc_auc_score(y_test, results))

0.9406559270938696


#### Вторая модель. 

In [91]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [92]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [93]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 26s 1s/step - loss: 0.4413 - accuracy: 0.8485 - val_loss: 0.3046 - val_accuracy: 0.8557
Epoch 2/10
24/24 [==============================] - 24s 1s/step - loss: 0.2478 - accuracy: 0.8905 - val_loss: 0.2124 - val_accuracy: 0.9086


In [94]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 3s 233ms/step


In [95]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [96]:
results_tresh = (results > 0.5) * 1

In [97]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.67      0.77      0.72      1029
           1       0.96      0.93      0.94      5488

    accuracy                           0.90      6517
   macro avg       0.81      0.85      0.83      6517
weighted avg       0.91      0.90      0.91      6517



In [98]:
print(roc_auc_score(y_test, results))

0.9422779836632696


#### Возьмем предобученный слой ембедингов

model.txt это скаченная с https://rusvectores.org/ru/ матрица эмбедингов на русские слова. Модель обучалась с дополнительными метками частей речи, но т.к. мы изначально их не использовали, то и отсюда удалим, чтоб мы могли использовать наш предобработанный текст.

In [102]:
import re

In [103]:
# Загрузка предобученной модели

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

In [104]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


#### Мы составили словарь типа слово:вектор эмбединга из файла модели. Теперь мы берем наш словарь который мы составили на основе наших текстов, и создаем матрицу весов. Т.е. берем слово в нашем словаре и по его индексу в словаре вставляем вектор эмбединга.

In [105]:
embedding_matrix = np.zeros((max_words, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [106]:
len(embedding_matrix)

10000

#### Теперь создаем сетку, и в качестве матрицы весов эмбединга задаем подготовленную матрицу. В первом варианте мы не будем дообучать наш эмбединг, во втором будем дообучать.

In [107]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [108]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [109]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 20s 758ms/step - loss: 0.4360 - accuracy: 0.8400 - val_loss: 0.2568 - val_accuracy: 0.8867
Epoch 2/10
24/24 [==============================] - 17s 726ms/step - loss: 0.2243 - accuracy: 0.8970 - val_loss: 0.2020 - val_accuracy: 0.9101


In [110]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 3s 190ms/step


In [111]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [112]:
results_tresh = (results > 0.5) * 1

In [113]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.70      0.70      0.70      1029
           1       0.94      0.94      0.94      5488

    accuracy                           0.91      6517
   macro avg       0.82      0.82      0.82      6517
weighted avg       0.91      0.91      0.91      6517



In [114]:
print(roc_auc_score(y_test, results))

0.9448229833374414


#### Второй вариант

In [115]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_len))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [116]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [117]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 26s 1s/step - loss: 0.4626 - accuracy: 0.8255 - val_loss: 0.2857 - val_accuracy: 0.8557
Epoch 2/10
24/24 [==============================] - 24s 1s/step - loss: 0.2324 - accuracy: 0.8913 - val_loss: 0.2025 - val_accuracy: 0.9139


In [118]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 3s 236ms/step


In [119]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [120]:
results_tresh = (results > 0.5) * 1

In [121]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.69      0.75      0.72      1029
           1       0.95      0.94      0.94      5488

    accuracy                           0.91      6517
   macro avg       0.82      0.84      0.83      6517
weighted avg       0.91      0.91      0.91      6517



In [122]:
print(roc_auc_score(y_test, results))

0.9483576854315238


#### В Целом если не дообучать эмбединг, то лучше результат на f1 score, да и вообще лучший результат. Если включить дообучение, то ввиду дисбаланса классов, модель работает хуже.